Import Libraries

In [44]:
import pandas as pd
import numpy as np
import os
import yfinance as yf


Currency Codes

In [45]:
country_currency_codes_table = {
    'Burundi': 'BIF',
    'Benin': 'XOF',
    'Burkina Faso': 'XOF',
    'Central African Republic': 'XAF',
    "Côte d'Ivoire": 'XOF',
    'Cameroon': 'XAF',
    'Democratic Republic of the Congo': 'CDF',
    'Republic of Congo': 'XAF',
    'Comoros': 'KMF',
    'Eritrea': 'ERN',
    'Ethiopia': 'ETB',
    'Ghana': 'GHS',
    'Guinea': 'GNF',
    'The Gambia': 'GMD',
    'Guinea-Bissau': 'XOF',
    'Liberia': 'LRD',
    'Madagascar': 'MGA',
    'Mali': 'XOF',
    'Mozambique': 'MZN',
    'Mauritania': 'MRU',
    'Malawi': 'MWK',
    'Niger': 'XOF',
    'Rwanda': 'RWF',
    'Sudan': 'SDG',
    'Senegal': 'XOF',
    'Sierra Leone': 'SLL',
    'Somalia': 'SOS',
    'Sao Tomee and Principe': 'STN',
    'Chad': 'XAF',
    'Togo': 'XOF',
    'Tanzania': 'TZS',
    'Uganda': 'UGX',
    'Zambia': 'ZMW'
}


Read Data

In [46]:
#read in stata dataset as dataframe "country_code_df"
country_code_df = pd.read_stata("codesffactors_Mar23_2019.dta") 
country_code_df.replace('--', np.nan, inplace=True) #replace all entries that are '--' with NaN data value

country_code_df.head()

,countryname,wbctry,wbcode,wbregion,wbigroup2017,wblcat2017,hipc,emu,cemac,waemu,imfcode,imfweocode,imfweoiso,imfweoctry,region
0,Aruba,Aruba,ABW,Latin America & Caribbean,High income,NaN,0,0,0,0,NaN,314.0,ABW,Aruba,LAC
1,Afghanistan,Afghanistan,AFG,South Asia,Low income,IDA,1,0,0,0,512.0,512.0,AFG,Afghanistan,SAS
2,Angola,Angola,AGO,Sub-Saharan Africa,Lower middle income,IBRD,0,0,0,0,614.0,614.0,AGO,Angola,SSA
3,Anguilla,NaN,AIA,Latin America & Caribbean,NaN,NaN,0,0,0,0,312.0,NaN,NaN,NaN,LAC
4,Albania,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,0,0,0,0,914.0,914.0,ALB,Albania,ECA


In [47]:
region_classification_ls = country_code_df['wbregion'].unique() #get a list of region classifications
income_classification_ls = country_code_df['wbigroup2017'].unique() #get a list of income classifications
income_classification_ls = [x for x in income_classification_ls if not pd.isna(x)] # remove '--' from income_classification_ls

print(f"The region classifications are: {', '.join(country for country in region_classification_ls)}")
print(f"The income classifications are: {', '.join(country for country in income_classification_ls)}")

The region classifications are: Latin America & Caribbean, South Asia, Sub-Saharan Africa, Europe & Central Asia, Middle East & North Africa, East Asia & Pacific, North America
The income classifications are: High income, Low income, Lower middle income, Upper middle income


In [48]:
ssa_hipc_countries = country_code_df[(country_code_df['hipc'] == 1) & (country_code_df['wbregion'] == 'Sub-Saharan Africa')]
ssa_hipc_countries_ls = ssa_hipc_countries['imfweoctry'].to_list()

ssa_hipc_countries_ls

['Burundi',
 'Benin',
 'Burkina Faso',
 'Central African Republic',
 "Côte d'Ivoire",
 'Cameroon',
 'Democratic Republic of the Congo',
 'Republic of Congo',
 'Comoros',
 'Eritrea',
 'Ethiopia',
 'Ghana',
 'Guinea',
 'The Gambia',
 'Guinea-Bissau',
 'Liberia',
 'Madagascar',
 'Mali',
 'Mozambique',
 'Mauritania',
 'Malawi',
 'Niger',
 'Rwanda',
 'Sudan',
 'Senegal',
 'Sierra Leone',
 'Somalia',
 'São Tomé and Príncipe',
 'Chad',
 'Togo',
 'Tanzania',
 'Uganda',
 'Zambia']

Read in IMF's Government Finance Statistics

In [49]:
master_table = {}
folder_path = "expenditure-investment-data"

# Loop that goes through all files 'file_name' in the folder 'folder_path'
for file_name in os.listdir(folder_path):
  if file_name.endswith('.xlsx') and not file_name.startswith('~$'): #check to see if is spreadsheet .xlsx format
    raw_df = pd.read_excel(folder_path+ "/" + file_name) #read in .xlsx as dataframe
    master_table[file_name[:-5]] = raw_df #store dataframe as an entry in the master_table hash table

# master_table["Central African Republic"]
for key in master_table:
  print(master_table[key])

                                           Unnamed: 0   Unnamed: 1  \
0   Statement of Operations (Revenue, Expenditures...          NaN   
1                                      Country: Benin          NaN   
2                Sector: Budgetary central government          NaN   
3                             Unit: Domestic currency          NaN   
4                                     Scale: Billions          NaN   
5                                                 NaN          NaN   
6                                                 NaN  2010.000000   
7                    TRANSACTIONS AFFECTING NET WORTH          NaN   
8                                             Revenue   648.375468   
9                                               Taxes   536.250675   
10                               Social contributions    16.366616   
11                                             Grants    52.312694   
12                                      Other revenue    43.445484   
13                  

Filtering

In [50]:
#REMOVE "JUNK" ROWS AND CONVERT TO CORRECT SCALE (MILLION/BILLIONS)
for key in master_table: #loop through table (where key is the country name) (master_table[key] is the dataframe stored in the master_table indexed by key)
  scale = master_table[key].iloc[4,0] #get if values have scale of billions or millions
  
  # Drop rows 0 to 5, reformatting indexes and column headers
  master_table[key] = master_table[key].drop([0, 1, 2, 3, 4, 5, 11]) 
  master_table[key] = master_table[key].reset_index(drop=True) 
  master_table[key].columns = master_table[key].iloc[0]
  master_table[key] = master_table[key][1:]
  master_table[key] = master_table[key].set_index(master_table[key].columns[0])
  master_table[key].columns = [int(float(col)) for col in master_table[key].columns]
  # master_table[key] = master_table[key].rename({4: 'Tax Grants'}, axis='index')
  # index_to_rename = master_table[key].index[master_table[key]['name'] == 'grants'][0]
  # master_table[key].at[index_to_rename, 'name'] = 'tax grants'


  # master_table[key] = master_table[key].fillna(0)

  if scale == "Scale: Billions":
    master_table[key] = master_table[key]* (10 ** 9) #multiple every value in dataframe by 1e9 if scale is billion
  elif scale == "Scale: Millions":
    master_table[key] = master_table[key]* (10 ** 6) #multiple every value in dataframe by 1e9 if scale is billion


#FILTER OUT YEAR COLUMNS ACCOUNTING DOESNT ADD UP
#first level: make sure 'Taxes', 'Social Contributions', 'Grants, 'Other Revenue' Sums to 'Revenue'
#second level: make sure 'Taxes on income, profits, & capital gains', 'Taxes on payroll & workforce', 'Taxes on property', 'Taxes on goods & services', 'Taxes on int trade & transactions', 'Other taxes n.e.c.' sum to 'Taxes'
first_level_ls = ['Compensation of employees', 'Use of goods and services', 'Consumption of fixed capital', 'Interest', 'Subsidies', 'Grants', 'Social benefits', 'Other expense', 'Net/gross investment in nonfinancial assets']
# second_level_ls = ['Taxes on income, profits, & capital gains', 'Taxes on payroll & workforce', 'Taxes on property', 'Taxes on goods & services', 'Taxes on int trade & transactions', 'Other taxes n.e.c.']
                  
for key in master_table:
  years_accounting_error_ls = []
  years_available = master_table[key].columns
  for year in years_available:
    country_year_df = master_table[key][year]
    # revenue = 0 
    exp = 0
    for category in first_level_ls:
      if category in country_year_df.index:
        exp += country_year_df[category]
      # else: 
      #   print(category)
    # for category in second_level_ls:
    #   if category in country_year_df.index:
    #     taxes += country_year_df[category]
    # if round(taxes, 0) != round(country_year_df['Taxes'], 0) or round(revenue, 0) != round(country_year_df['Revenue'], 0):
    
    margin = 0.001
    lower_bound_exp = country_year_df['Expenditure'] - country_year_df['Expenditure']*margin
    upper_bound_exp = country_year_df['Expenditure'] + country_year_df['Expenditure']*margin

    # print(lower_bound_exp)
    # print(upper_bound_exp)
    # print(exp)
    if exp < lower_bound_exp or exp > upper_bound_exp:
    
      print(f"{round(exp, 0)}, {round(country_year_df['exp'], 0)}")

      years_accounting_error_ls.append(year)
  
  print(key)
  print(years_accounting_error_ls)
  master_table[key].drop(columns=years_accounting_error_ls, inplace=True)
 
    
    

#FILTER OUT COUNTRIES WITH LESS THAN 7 YEARS OF DATA
less_than_seven_years_data_countries = [] #create list to store countries with less than 7 years of historial data
for key in master_table: #iterate through master_table
  num_columns = master_table[key].shape[1] #count the number of columns (each column represents a year)
  if num_columns < 7: #if less than 7 columns/year, add to list to delete
    less_than_seven_years_data_countries.append(key)

print('remove')
for country in less_than_seven_years_data_countries:
  print(country)
  del master_table[country] #delete countries with less than 7 years from data table

master_table["Zambia"]

Benin
[]
Senegal
[]
Côte d'Ivoire
[]
Burkina Faso
[]
Central African Republic
[]
Ghana
[]
Somalia
[]
Tanzania
[]
Zambia
[]
Democratic Republic of the Congo
[]
Rwanda
[]
Republic of Congo
[]
Sudan
[]
The Gambia
[]
Togo
[]
Mali
[]
Malawi
[]
Sao Tomee and Principe
[]
Mozambique
[]
Sierra Leone
[]
Guinea-Bissau
[]
Ethiopia
[]
Burundi
[]
Liberia
[]
Cameroon
[]
Madagascar
[]
Uganda
[]
remove
Benin
Somalia
The Gambia
Guinea-Bissau
Liberia


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
nan,,,,,,,,,,
TRANSACTIONS AFFECTING NET WORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,1.612499e+10,2.401304e+10,2.403718e+10,2.678465e+10,3.128051e+10,3.641924e+10,4.283610e+10,4.870935e+10,5.619082e+10,6.231981e+10
Taxes,1.247751e+10,1.678054e+10,1.954949e+10,2.171162e+10,2.632927e+10,2.639580e+10,2.885596e+10,3.739301e+10,4.564338e+10,5.010771e+10
Social contributions,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Other revenue,1.485211e+09,3.138591e+09,2.331028e+09,2.858794e+09,3.301976e+09,7.155174e+09,1.032787e+10,4.103434e+09,7.770645e+09,8.768542e+09
Expense,1.426046e+10,1.652937e+10,2.170219e+10,2.701759e+10,3.383866e+10,4.260635e+10,4.842472e+10,5.152103e+10,5.533955e+10,6.596701e+10
Compensation of employees,6.394299e+09,6.950181e+09,1.066433e+10,1.137620e+10,1.536678e+10,1.639741e+10,1.864327e+10,2.002802e+10,2.096917e+10,2.233316e+10
Use of goods and services,3.440688e+09,6.507450e+09,5.394747e+09,7.762790e+09,7.342421e+09,8.956639e+09,1.425967e+10,1.244033e+10,8.807746e+09,9.273553e+09
Consumption of fixed capital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Convert Currency

In [51]:
print('keep')
for key in master_table:
  print(key)

#CONVERT LOCAL CURRENCY DENOMINATED VALUES TO USD
error_countries = []
for key in master_table:
  #get annual 2010-2019 USD to domestic currency exchange rate using yfinance library
  currency = country_currency_codes_table[key] #retrieve currency symbol from 'country_currency_codes_table'
  symbol = f"{currency}=X"
  try:
    data = yf.download(symbol, start="2010-01-01", end="2020-01-01", interval="1mo") #download monthly USD-Currency dataset from 2010-2020
    yearly_data_df = data['Close'].resample('YE').mean() #get the annual average exchange rate outputted as datafram
    yearly_data_table = yearly_data_df.to_dict() # Convert the yearly_data to a hash table with the date as the key
    year_to_exchange_rate_dict = {key.year: value for key, value in yearly_data_table.items()} # Convert the keys to only the year part
    print(year_to_exchange_rate_dict)

    #convert DataFrame values to USD
    for year in master_table[key].columns: #loop through columns in dataframe
      year_int = int(year)  # Convert year to integer to match the dictionary keys
      master_table[key][year] = master_table[key][year] / year_to_exchange_rate_dict[year_int] #divide each column's values by that year's exchange rate

  except Exception as e: #if errored out
    print(f"Could not fetch rate for {key} ({currency}): {e}")
    error_countries.append(key) #add country to error list
    pass
    
for country in error_countries:
  del master_table[country] #delete countries where currency conversion failed
 
for key in master_table:
  print(key)
  print(master_table[key])

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

keep
Senegal
Côte d'Ivoire
Burkina Faso
Central African Republic
Ghana
Tanzania
Zambia
Democratic Republic of the Congo
Rwanda
Republic of Congo
Sudan
Togo
Mali
Malawi
Sao Tomee and Principe
Mozambique
Sierra Leone
Ethiopia
Burundi
Cameroon
Madagascar
Uganda
{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

{2010: 487.5991668701172, 2011: 460.59083048502606, 2012: 501.04666900634766, 2013: 492.91916910807294, 2014: 497.4808349609375, 2015: 594.4341735839844, 2016: 594.0333353678385, 2017: 578.2424977620443, 2018: 556.1933390299479, 2019: 586.7139180501302}
{2010: 1.420966664950053, 2011: 1.5373833378156025, 2012: 1.8385500013828278, 2013: 2.0669749677181244, 2014: 3.0854166547457376, 2015: 3.806433320045471, 2016: 3.9582666556040444, 2017: 4.381908297538757, 2018: 4.706224997838338, 2019: 5.369808276494344}
{2010: 1422.5833231608074, 2011: 1568.4333394368489, 2012: 1563.4083455403645, 2013: 1581.8916727701824, 2014: 1632.8416646321614, 2015: 1989.9833170572917, 2016: 2136.6749674479165, 2017: 2193.6416625976562, 2018: 2273.074991861979, 2019: 2301.0916748046875}
{2013: 5.374436378479004, 2014: 6.177958369255066, 2015: 8.771591663360596, 2016: 10.236883401870728, 2017: 9.491416613260904, 2018: 10.564791679382324, 2019: 13.000277121861776}
Could not fetch rate for Zambia (ZMW): 2010
{2010: 

...

{2010: 2.318599979082743, 2011: 2.648924986521403, 2012: 3.52014168103536, 2013: 4.590566674868266, 2014: 5.666366696357727, 2015: 5.987816691398621, 2016: 6.130283355712891, 2017: 6.609750032424927, 2018: 25.39816665649414, 2019: 45.693160692850746}



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['STN=X']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 148.9291674296061, 2011: 155.58916854858398, 2012: 247.6258316040039, 2013: 362.4624989827474, 2014: 410.5541687011719, 2015: 495.21583557128906, 2016: 700.3816680908203, 2017: 720.1374969482422, 2018: 704.3308308919271, 2019: 718.8183339436849}
Could not fetch rate for Sao Tomee and Principe (STN): Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


{2010: 34.550000508626304, 2011: 28.579166730244953, 2012: 28.048333326975506, 2013: 29.664999802907307, 2014: 31.18499978383382, 2015: 39.24833265940348, 2016: 62.973334312438965, 2017: 62.673333168029785, 2018: 59.819166819254555, 2019: 61.90666675567627}
{2010: 3920.0, 2011: 4280.166666666667, 2012: 4277.166666666667, 2013: 4268.166666666667, 2014: 4302.333333333333, 2015: 4348.583333333333, 2016: 5087.916666666667, 2017: 7505.0, 2018: 7962.916666666667, 2019: 9048.333333333334}
{2010: 14.29200005531311, 2011: 16.807833194732666, 2012: 17.636333147684734, 2013: 18.556583563486736, 2014: 19.54866663614909, 2015: 20.516416549682617, 2016: 21.691750208536785, 2017: 23.84641679128011, 2018: 27.412750085194904, 2019: 29.055416584014893}
{2010: 1206.0750122070312, 2011: 1216.6916605631511, 2012: 1396.4749857584636, 2013: 1531.5916748046875, 2014: 1530.9249877929688, 2015: 1536.9166666666667, 2016: 1605.7583312988281, 2017: 1700.5, 2018: 1756.2916666666667, 2019: 1822.7166646321614}
{2010:

Calculate Yearly Averages

In [52]:
data_by_year_table = {
  2010: [],
  2011: [],
  2012: [],
  2013: [],
  2014: [],
  2015: [],
  2016: [],
  2017: [],
  2018: [],
  2019: [],
} #make table that stores data by year

# master_averages_ls = [] #make master averages list as place to store averages

for key in master_table:
  years_available = master_table[key].columns
  for year in data_by_year_table:
    if year in years_available:
      data_by_year_table[year].append(master_table[key][year])

yearly_averages_ls = [] #make a list of the yearly averages
for year in data_by_year_table:
  merged_year_df = pd.concat(data_by_year_table[year], axis=1, join='outer')
  #SUM OR MEAN?
  year_average_df = merged_year_df.sum(axis=1) #get the average of all the countries' averages
  yearly_averages_ls.append(year_average_df)

 
# #Get average over decade
# #loop through table (where key is the country name) (master_table[key] is the dataframe stored in the master_table indexed by key)
# for key in master_table:
#   per_category_average = master_table[key].mean(axis=1) #get the average of each category (ignoring np.NaN values) into a list
#   # per_category_average.drop('To other government units', inplace=True) #drop 'To other government units' since there's two rows named that and will error
#   master_averages_ls.append(per_category_average) #add each country average to to the master averages list



# merged_averages_df = pd.concat(master_averages_ls, axis=1, join='outer')
# average__of_averages_df = merged_averages_df.mean(axis=1) #get the average of all the countries' averages
# average__of_averages_df

for i in range(len(yearly_averages_ls)):
  print(f"YEAR: {2010+i}")
  print(yearly_averages_ls[i])

YEAR: 2010
nan
TRANSACTIONS AFFECTING NET WORTH                                0.000000e+00
Revenue                                                         5.911121e+10
Taxes                                                           3.280798e+10
Social contributions                                            1.159401e+08
Other revenue                                                   1.268427e+10
Expense                                                         4.558989e+10
Compensation of employees                                       1.427636e+10
Use of goods and services                                       9.118777e+09
Consumption of fixed capital                                    0.000000e+00
Interest                                                        3.453300e+09
Subsidies                                                       3.144477e+09
Grants                                                          1.187730e+10
Social benefits                                              

Calculate Percentages

In [53]:
yearly_averages_percentage_ls = []

year = 2010
for year_average_df in yearly_averages_ls:


  total_taxes = year_average_df['Expenditure']
  year_average_percentage_df = year_average_df / total_taxes * 100 #take all values as a fraction of total expenditure, then convert decimal to percent, then round to two decimal places
  
  first_level = 0
  print(year)
  for category in first_level_ls:
    first_level += year_average_percentage_df[category]
  print(first_level)
  year += 1
  
  yearly_averages_percentage_ls.append(year_average_percentage_df)

for i in range(len(yearly_averages_percentage_ls)):
  print(f"YEAR: {2010+i}")
  print(yearly_averages_percentage_ls[i])

2010
98.52618780767818
2011
98.60935535641265
2012
96.3128972705741
2013
102.09813611332588
2014
97.19351604148883
2015
97.09045412203399
2016
97.57296441241434
2017
99.71824427649575
2018
99.99999999999997
2019
100.0
YEAR: 2010
nan
TRANSACTIONS AFFECTING NET WORTH                                  0.000000
Revenue                                                          93.848922
Taxes                                                            52.088155
Social contributions                                              0.184074
Other revenue                                                    20.138398
Expense                                                          72.381562
Compensation of employees                                        22.666102
Use of goods and services                                        14.477581
Consumption of fixed capital                                      0.000000
Interest                                                          5.482691
Subsidies        

Calculate Average Across Period

In [54]:
merged_percentages_df = pd.concat(yearly_averages_percentage_ls, axis=1, join='outer')
average_df = merged_percentages_df.mean(axis=1)
rounded_average_df = average_df.apply(lambda x: round(x, 2))

rounded_average_df = rounded_average_df.to_frame()
rounded_average_df

,0
nan,
TRANSACTIONS AFFECTING NET WORTH,0.00
Revenue,90.47
Taxes,63.22
Social contributions,0.26
Other revenue,15.07
Expense,76.60
Compensation of employees,24.31
Use of goods and services,15.00
Consumption of fixed capital,0.00


In [55]:
#check
first_level = 0
for category in first_level_ls:
  first_level += average_df[category]
print(first_level)

# second_level = 0
# for category in second_level_ls:
#   second_level += average_df[category]
# print(second_level)

98.71217554004238


In [57]:
rounded_by_year_average_df = merged_percentages_df.round(2)
rounded_by_year_average_df

,0,1,2,3,4,5,6,7,8,9
nan,,,,,,,,,,
TRANSACTIONS AFFECTING NET WORTH,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Revenue,93.85,109.75,86.80,97.59,91.43,90.32,81.41,82.65,85.11,85.83
Taxes,52.09,64.90,59.20,62.00,64.70,67.65,64.15,65.41,65.85,66.22
Social contributions,0.18,0.36,0.22,0.27,0.26,0.26,0.19,0.27,0.35,0.22
Other revenue,20.14,27.53,16.88,17.61,15.09,11.71,9.19,10.04,11.63,10.88
Expense,72.38,72.21,75.78,76.95,74.08,78.15,78.11,78.96,80.85,78.53
Compensation of employees,22.67,22.96,25.35,25.66,23.15,24.22,25.16,24.02,24.51,25.43
Use of goods and services,14.48,17.13,14.23,14.56,15.58,15.47,14.98,13.70,15.11,14.75
Consumption of fixed capital,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
